In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json
from underthesea import word_tokenize

from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import datasets
device = "cuda" if torch.cuda.is_available() else "cpu"


import re

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from tqdm.notebook import tqdm

In [11]:
class config:
    train_path = 'data/train.csv'
    model_name = 'vinai/phobert-base-v2'
    max_length = 256
    batch_size = 64
    num_workers = 12

In [4]:
df = pd.read_csv(config.train_path)
sample = df.sample(1)

In [5]:
sample['top_tfdif'].values

array(['ông cho biết , từ trao_đổi thực_tế với các doanh_nghiệp sản_xuất , deep signature đã hoàn_thiện bộ công_cụ cho_phép bất_kỳ doanh_nghiệp hay cá_nhân có_thể tạo và tự in hàng ngàn mã chống hàng giả cho sản_phẩm của mình với chi_phí rẻ hơn nhiều lần những giải_pháp hiện_nay trên thị_trường \nanh hồng chia_sẻ sau cuộc thi họ chưa có nhiều đơn hàng mới do các nguyên_nhân khách_quan từ thực_tế như giá xăng_dầu tăng , có thời_điểm người_dân hạn_chế đi biển \ntác_giả được giải sẽ nhận phần_thưởng là tiền_mặt và các lợi_ích khác như cơ_hội được truyền_thông trên các nền_tảng của báo_điện_tử vnexpress , được kết_nối tới các đối_tác quan_tâm để phát_triển , hoàn_thiện , kinh_doanh sản_phẩm \nsong anh cho biết nhóm tiếp_tục nỗ_lực hoàn_thiện hơn sản_phẩm và đợi thời_gian tới để triển_khai \nđây là năm thứ 2 cuộc thi sáng_kiến khoa_học ( creative science contest ) được tổ_chức , kỳ_vọng khuyến_khích phong_trào nghiên_cứu khoa_học , tạo động_lực cho việc phát_triển , ứng_dụng các sáng_kiến p

In [6]:
df_train, df_val = train_test_split(df, test_size=0.2, random_state=42)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [7]:
df_train.shape, df_val.shape

((30056, 9), (7515, 9))

In [8]:
train_dataset = datasets.Dataset.from_pandas(df_train)
val_dataset = datasets.Dataset.from_pandas(df_val)

In [9]:
label2id = {'SUPPORTED': 0, 'REFUTED': 1,'NEI': 2}
id2label = {0: 'SUPPORTED', 1:'REFUTED', 2:'NEI'}

In [12]:
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
model = AutoModelForSequenceClassification.from_pretrained(config.model_name, num_labels=3, id2label=id2label, label2id=label2id).to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def PreprocessDataset(examples):    
    inputs =  tokenizer(
            text=examples['claim_tokenizer'],
            text_pair=examples['top_tfdif'],
            max_length=config.max_length,
            padding='max_length',
            truncation='only_second',
            return_tensors='pt',
            
        )
    labels = examples['verdict_label']
    inputs.update({'labels': labels})
    return inputs

In [14]:
train_datasets = train_dataset.map(PreprocessDataset, batched=True, batch_size=64,remove_columns=train_dataset.column_names)
valid_datasets = val_dataset.map(PreprocessDataset, batched=True,  batch_size=64,remove_columns=train_dataset.column_names)

Map:   0%|          | 0/30056 [00:00<?, ? examples/s]

Map:   0%|          | 0/7515 [00:00<?, ? examples/s]

In [15]:
train_datasets, valid_datasets

(Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
     num_rows: 30056
 }),
 Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
     num_rows: 7515
 }))

In [16]:
len(train_datasets[0]['input_ids'])

256

In [17]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='micro')  # Có thể thay 'weighted' bằng 'micro', 'macro', hoặc None tùy vào yêu cầu của bạn
    
    return {
        'accuracy': accuracy,
        'f1_score': f1
    }


In [19]:
training_args = TrainingArguments(
    output_dir='models/model_v7',          # output directory
    num_train_epochs=20,              # total number of training epochs
    learning_rate= 1e-5,              # learning rate
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    # gradient_accumulation_steps=2,   # Number of updates steps to accumulate before performing a backward/update pass.
    warmup_steps=250,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=250,
    eval_steps=250,
    evaluation_strategy='steps',
    load_best_model_at_end=True,
    metric_for_best_model='f1_score',
    greater_is_better=True,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_datasets,
    eval_dataset=valid_datasets,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

/media/bbsw/Data/Hung-ws/lazy/v2/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy,F1 Score
250,0.942400,0.689988,0.723353,0.723353
500,0.558200,0.442229,0.839787,0.839787
750,0.392400,0.384370,0.860812,0.860812
1000,0.312600,0.369140,0.875316,0.875316


/media/bbsw/Data/Hung-ws/lazy/v2/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/media/bbsw/Data/Hung-ws/lazy/v2/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/media/bbsw/Data/Hung-ws/lazy/v2/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/media/bbsw/Data/Hung-ws/lazy/v2/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functio

KeyboardInterrupt: 

In [20]:
trainer.evaluate()

/media/bbsw/Data/Hung-ws/lazy/v2/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.0991379022598267,
 'eval_accuracy': 0.3473053892215569,
 'eval_f1_score': 0.3473053892215569,
 'eval_runtime': 19.4018,
 'eval_samples_per_second': 387.334,
 'eval_steps_per_second': 6.082}

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('finals/model_v3')